### import functions

In [1]:
from sklearn.model_selection  import cross_val_score, train_test_split, cross_validate, cross_val_score,KFold,RandomizedSearchCV,RepeatedKFold, GridSearchCV,StratifiedKFold
from sklearn import svm,tree, neighbors,linear_model, preprocessing
from sklearn.metrics import make_scorer, accuracy_score, confusion_matrix, classification_report, roc_curve, auc, r2_score
from sklearn import metrics
import xgboost as xgb
import numpy as np
import pandas as pd 
from sklearn.model_selection  import cross_val_score, train_test_split, cross_validate, cross_val_score,KFold,RandomizedSearchCV,RepeatedKFold, GridSearchCV,StratifiedKFold

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 
import missingno as msno 

### load in data

In [ ]:
app_train = pd.read_csv('C:/Users/yidan/Desktop/PA/Project/application_train.csv')
app_test = pd.read_csv('C:/Users/yidan/Desktop/PA/Project/application_test.csv')
bureau= pd.read_csv('C:/Users/yidan/Desktop/PA/Project/bureau.csv')
b_balance= pd.read_csv('C:/Users/yidan/Desktop/PA/Project/bureau_balance.csv')
c_balance= pd.read_csv('C:/Users/yidan/Desktop/PA/Project/credit_card_balance.csv')
install= pd.read_csv('C:/Users/yidan/Desktop/PA/Project/installments_payments.csv')

### categorical -> numerical

In [13]:
#categorical variable with less than 
# 3 unique values, encode to 0,1,2
le = LabelEncoder()
le_count = 0

In [14]:
# Iterate through the columns
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

3 columns were label encoded.


In [16]:
#categorical variable with more than 
# 3 unique values, convert to dummy variables
# one-hot encoding of categorical variables
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

Training Features shape:  (307511, 243)
Testing Features shape:  (48744, 239)


In [17]:
app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

print('There are %d anomalies in the test data out of %d entries' % (app_test["DAYS_EMPLOYED_ANOM"].sum(), len(app_test)))

There are 9274 anomalies in the test data out of 48744 entries


### Label Y

In [18]:
train_labels = app_train['TARGET']

# Align the training and testing data, keep only columns present in both dataframes
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)

# Add the target back in
app_train['TARGET'] = train_labels

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

Training Features shape:  (307511, 240)
Testing Features shape:  (48744, 239)


### impute missing values

In [19]:
#filling in the missing values (imputation) and 
#normalizing the range of the features (feature scaling)
from sklearn.preprocessing import MinMaxScaler, Imputer

# Drop the target from the training data
if 'TARGET' in app_train:
    train = app_train.drop(columns = ['TARGET'])
else:
    train = app_train.copy()
    
# Feature names
features = list(train.columns)

# Copy of the testing data
test = app_test.copy()

# Median imputation of missing values
imputer = Imputer(strategy = 'median')

# Scale each feature to 0-1
scaler = MinMaxScaler(feature_range = (0, 1))

In [20]:
# Fit on the training data
imputer.fit(train)

# Transform both training and testing data
train = imputer.transform(train)
test = imputer.transform(app_test)

In [21]:
# Repeat with the scaler
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

print('Training data shape: ', train.shape)
print('Testing data shape: ', test.shape)

Training data shape:  (307511, 239)
Testing data shape:  (48744, 239)


### split train test

In [22]:
X_train, X_test, y_train, y_test = train_test_split(
    train, train_labels, train_size=0.8)

### Logistic Reg

In [ ]:
from sklearn.linear_model import LogisticRegression

# Make the model with the specified regularization parameter
log_reg = LogisticRegression(C = 0.0001)

# Train on the training data
log_reg.fit(train, train_labels)

k_fold = StratifiedKFold(n_splits=2)
clf_log = linear_model.LogisticRegression(solver='liblinear')

param_range = [0.01,0.1, 1, 10, 100]
params= {'C': param_range, 'penalty': ['l1','l2']}
scoring_fnc = make_scorer(accuracy_score)
grid = GridSearchCV(clf_log, param_grid=params,scoring=scoring_fnc,cv=k_fold)
grid = grid.fit(X_train, y_train)
clf_log = grid.best_estimator_
r2 = r2_score(y_test, grid.predict(X_test))

print('Logistic Regression R2: %f'%r2)
print('best parameters:')
for key in params.keys():
    print('%s: %s'%(key, clf_log.get_params()[key]))
print('test score: %f'%clf_log.score(X_test, y_test))
print(classification_report(y_test,grid.predict(X_test)))

### Random Forest

In [47]:
#Random Forest
from sklearn import ensemble
clf_rf=ensemble.RandomForestClassifier()

k_fold=KFold(n_splits=2)
params= {'n_estimators': range(1,20), 'max_depth': range(1,20)}
scoring_fnc = make_scorer(accuracy_score)
grid = GridSearchCV(clf_rf, param_grid=params,scoring=scoring_fnc,cv=k_fold)
grid = grid.fit(X_train, y_train)
clf_rand= grid.best_estimator_
r2 = r2_score(y_test, grid.predict(X_test))

print('Random Forest R2: %f'%r2)
print('best parameters:')
for key in params.keys():
    print('%s: %s'%(key, clf_rf.get_params()[key]))
print('test Accuracy score: %f'%clf_rand.score(X_test, y_test))
print(classification_report(y_test,grid.predict(X_test)))

Random Forest R2: -0.089111
best parameters:
n_estimators: warn
max_depth: None
test Accuracy score: 0.918291
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     84717
           1       0.33      0.00      0.00      7537

    accuracy                           0.92     92254
   macro avg       0.63      0.50      0.48     92254
weighted avg       0.87      0.92      0.88     92254



### XGBoost

In [2]:
#XGBoost
import xgboost as xgb
data_dmatrix = xgb.DMatrix(data=train,label=train_labels)
X_train, X_test, y_train, y_test = train_test_split(train, train_labels, test_size=0.3)
xg_reg = xgb.XGBRegressor()

params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}
cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=2,metrics="auc", num_boost_round=50,early_stopping_rounds=10,as_pandas=True, seed=123)


xg_reg = xg_reg.fit(X_train,y_train)
mse = metrics.mean_squared_error(y_test, xg_reg.predict(X_test))
r2 = metrics.r2_score(y_test, xg_reg.predict(X_test))
clf_xgb = xg_reg.best_estimator_
#print('best parameters:')
#for key in params.keys():
#    print('%s: %s'%(key, clf_xgb.get_params()[key]))
print(f'XGBoost MSE:{mse}')
print(f'XGBoost r-squared:{r2}')
print('test AUC score: %f'%clf_xgb.score(X_test, y_test))
print(classification_report(y_test,grid.predict(X_test)))

In [40]:
import xgboost as xgb
data_dmatrix = xgb.DMatrix(data=train,label=train_labels)
X_train, X_test, y_train, y_test = train_test_split(train, train_labels, test_size=0.3)
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

res=xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)

mse = metrics.mean_squared_error(y_test, res.predict(X_test))
r2 = metrics.r2_score(y_test, res.predict(X_test))
print(xg_reg)


[21:25:47] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=None, n_estimators=10,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)
XGBoost MSE:0.09293329288314628
XGBoost R2:-0.23871858492968245


In [41]:
print(f'XGBoost MSE:{mse}')
print(f'XGBoost R2:{r2}')
print('test AUC score: %f'%xg_reg.score(X_test, y_test))

XGBoost MSE:0.09293329288314628
XGBoost R2:-0.23871858492968245
test AUC score: -0.238719


In [44]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


### LightGBM

In [1]:
# random search lightGBM
import lightgbm as lgb
import ast
random_best = ast.literal_eval(random.loc[0, 'hyperparameters'])

rmodel = lgb.LGBMClassifier(**random_best)
rmodel.fit(train, train_labels)

In [ ]:
k_fold=KFold(n_splits=2)
scoring_fnc = make_scorer(accuracy_score)
grid = GridSearchCV(clf_log, param_grid=params,scoring=scoring_fnc,cv=k_fold)
grid = rmodel.fit(X_train, y_train)
clf_GBM = grid.best_estimator_
r2 = r2_score(y_test, grid.predict(X_test))

print('lightGBM R2: %f'%r2)
print('best parameters:')
for key in params.keys():
    print('%s: %s'%(key, clf_GBM.get_params()[key]))
print('test Accuracy score: %f'%clf_GBM.score(X_test, y_test))
print(classification_report(y_test,grid.predict(X_test)))

In [ ]:
# Bayesian Optimization lightGBM
bayes_best = ast.literal_eval(opt.loc[0, 'hyperparameters'])

bmodel = lgb.LGBMClassifier(**bayes_best)
bmodel.fit(train, train_labels)

In [ ]:
k_fold=KFold(n_splits=2)
scoring_fnc = make_scorer(accuracy_score)
grid = GridSearchCV(clf_log, param_grid=params,scoring=scoring_fnc,cv=k_fold)
grid = grid.fit(X_train, y_train)
clf_log = grid.best_estimator_
r2 = r2_score(y_test, grid.predict(X_test))

print('Linear Regression R2: %f'%r2)
print('best parameters:')
for key in params.keys():
    print('%s: %s'%(key, clf_log.get_params()[key]))
print('test Accuracy score: %f'%clf_log.score(X_test, y_test))
print(classification_report(y_test,grid.predict(X_test)))